# Setup

In [47]:
import os
import openai
import json
 
openai.api_key = os.environ["OPENAI_API_KEY"]

def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=1100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [103]:
subject_prompt = '''
Input::
미세먼지: 보통
초미세먼지: 좋음
날씨: 흐린날
시간대: 점심
요일: 화요일
계절: 봄
최저기온: 12도
최고기온: 17도
일교차: 적음
나이: 20
성별: 여성
직업: 대학생
관심사: 동네탐방, 맛집탐방
Output::
(미세먼지, 초미세먼지, 날씨, 시간대, 직업, 관심사) 점심 메뉴를 찾기 위한 학식 탐방
(미세먼지, 초미세먼지, 날씨, 시간대, 관심사) 막간의 동네 공원 산책
(미세먼지, 초미세먼지, 날씨, 시간대, 계절, 관심사) 봄철 동네 맛집 탐방
(미세먼지, 초미세먼지, 날씨, 시간대, 성별, 직업) 흐린 날씨도 산뜻하게 해줄 봄철 옷차림
(미세먼지, 초미세먼지, 날씨, 시간대, 관심사) 잠을 깨워줄 동네 작은 카페 탐방
Input:: 
{input}
Output::
'''

sentece_prompt = '''
Input:: 점심 메뉴를 찾기 위한 학식 탐방
Output::
1. 수업 듣느라 너무 힘들었어 ㅠ 오늘 학식 메뉴 뭐지?
2. 학식의 진리는 돈까스지!
Input:: 흐린 날씨도 산뜻하게 해줄 봄철 옷차림
Output::
1. 날씨는 흐리지만 산뜻한 원피스 입어볼까?
2. 밝은 색 옷을 입으면 꿀꿀한 기분도 사라질거야~
Input:: 여름밤의 실내 영화 감상
Output::
1. 여름밤엔 역시 OTT 핫리스트 영화지!
2. 더운 밤에 어딜 나가겠어? 시원한 내 방에서 영화 한편 보는 게 짱이지!
Input:: {input}
Output::
'''

value_prompt1 = '''
Divide Input into several parts and find the corresponding item from given Info for each part: if exists, give Item O, otherwise Item X.
If there is no Item X, Output will be Good, otherwise Bad.

Info::
요일: 화요일
계절: 봄
미세먼지: 보통
초미세먼지: 좋음
날씨: 흐린날
최저기온: 12도
최고기온: 17도
일교차: 적음
시간대: 점심
나이: 20
성별: 남성
직업: 대학생
관심사: 동네탐방, 맛집탐방
Input:: 수업 듣느라 너무 힘들었어 ㅠ 오늘 학식 메뉴 뭐지?
수업 듣느라 너무 힘들었어 ㅠ => 직업 O
오늘 학식 메뉴 뭐지? => 직업 O, 관심사 O
Output:: Good
Input:: 역시 저녁엔 빡센 운동을!!!
역시 저녁엔 => 시간대 X
빡센 운동을!!! => 관심사 X
Output:: Bad
Input:: 날씨는 흐리지만 산뜻한 원피스 입어볼까?
날씨는 흐리지만 => 날씨 X
산뜻한 원피스 입어볼까? => 성별 X
Output:: Bad
Input:: 날씨도 좋은데 우리 어디 나가자~
날씨도 좋은데 => 날씨 X
우리 어디 나가자~ => 관심사 O
Output:: Bad(날씨 흐림)

Info::
{info}
Input:: {input}
'''

value_prompt2 = '''
다음 두 문장 중 주제에 더 어울리는 문장을 문장 앞의 숫자로 골라주세요. 만약 어울림이 비슷하다면, "유사"라고 답변해 주세요. 

주제:: 추운 겨울날 캠핑을 해보자!
1. 겨울날 캠핑하기 좋은 장소는 어디일까?
2. 겨울날 캠핑하기 위해선 따뜻하고 보온성 있는 옷과 장비가 필수지!
Output:: 1

주제:: {subject}
{sentences}
'''

In [82]:
llm('''Find any "not coherence" parts from Input. If there exists, give Bad the part. Otherwise, give Good.
Meaning of "not coherent" 
- some entity or item having opposite propertiies in a sentence; e.g. 따뜻한 아이스 아메리카노
- some event occurring in an unusal circumstance; e.g. 나쁜 미세먼지에 야외 활동, 주말 근무 준비 

Input:: 여름밤의 영화관은 역시 시원하고 따뜻한 방문객이 많아!
Output:: Bad - 시원하고 따뜻한
Input:: 주말 아침에 든든한 식사로 업무 준비해 보자~
Output:: Bad - 주말 아침에 든든한 식사로 업무 준비
Input:: 따뜻한 아이스 아메리카노 주세요.
Output:: ''', stop=['Input::'])

' Good'

In [86]:
llm('''Input에서 일관성이 떨어지는 부분을 찾아보세요. 만약 그런부분이 존재한다면 Output에 Bad를 없다면 Good을 주세요.
"일관성이 떨어지는" 경우
- 문장 내에서 동일한 아이템이 반대되는 특성을 가진 경우; e.g. 따뜻한 아이스 아메리카노
- 특정 사건이 일반적이지 않은 상황에서 발생하는 경우; e.g. 나쁜 미세먼지에 야외 활동, 주말 근무 준비 

Input:: 여름밤의 영화관은 역시 시원하고 따뜻한 방문객이 많아!
Output:: Bad - 시원하고 따뜻한 방문객
Input:: 주말 아침에 든든한 식사로 업무 준비해 보자~
Output:: Bad - 주말 아침에 든든한 식사로 업무 준비
Input:: 덥고 추운 여름 장마
Output:: ''', stop=['Input::'])

' Good'

In [105]:
test_file = './gowun_data/profile.txt'
with open(test_file, 'r') as f:
    test_cases = f.read()
test_cases = test_cases.strip().split('\n\n')

outputs = []
for t in test_cases:
    print(t)
    full_str = t

    # Step1: 주어진 정보로 주제 뽑기
    subject_list = llm(subject_prompt.format(input=t), stop=["Input::"])
    subject_list = subject_list.split('\n')

    # Step2: 주제 별 문장 생성하기
    # Step3: 주제 별 생성된 두 문장 중에서 더 나은 문장 고르기
    tmp = []
    for i, s in enumerate(subject_list):
        subject = s[s.index(') ') + 1:]
        sentences = llm(sentece_prompt.format(input=subject), stop=["Input::"])
        answer = llm(value_prompt2.format(subject=subject, sentences=sentences), stop=["주제::"])
        tmp.append(sentences + '\n' + answer)
        print(subject_list[i])
        print(tmp[i])
        full_str += (f'\n{subject_list[i]}\n{tmp[i]}')
    
    outputs.append(full_str)

요일: 수요일
계절: 여름
미세먼지: 보통
초미세먼지: 좋음
날씨: 비
최저기온: 20도
최고기온: 30도
일교차: 큼
시간대: 저녁
나이: 26
성별: 여성
직업: 직장인
관심사: 금융/부동산 투자, 영화감상
(미세먼지, 초미세먼지, 날씨, 시간대, 성별, 관심사) 여름밤을 즐길 수 있는 영화감상 프로그램
1. 여름밤엔 시원한 영화관에서 실내 영화 감상하는 거 어때?
2. 밤하늘을 바라보며 실내 영화 감상하는 단체 밤하늘 영화관 프로그램 시도해보자!
Output:: 2
(미세먼지, 초미세먼지, 날씨, 시간대, 직업, 관심사) 비도 오고 날도 쌀쌀한 밤, 금융/부동산 투자를 위한 비교검토
1. 비도 오고 날도 쌀쌀한 밤, 금융/부동산 투자를 위해 시장 동향과 최신 정보를 찾아볼까?
2. 자신에게 맞는 투자를 찾기 위해 심사숙고를 해보는 건 어떨까?
Output:: 1
(미세먼지, 초미세먼지, 날씨, 시간대, 성별, 직업) 여름밤 덥고 습한 날씨를 식힐 수 있는 여름밤 메뉴
1. 여름밤엔 바나나 막대 아이스크림!
2. 여름밤 더위를 식히기 좋은 따뜻한 메뉴는 닭갈비 따뜻한 밥과 함께 하면 좋을 것 같아!
Output:: 2
(미세먼지, 초미세먼지, 날씨, 시간대, 관심사) 비도 오고 날도 쌀쌀한 밤, 명품 식당으로 여름밤을 마무리하기
1. 여름밤이라면 담백한 생선 요리를 먹어볼까?
2. 비도 오고 날도 추운 밤, 따뜻한 스파게티로 여름밤을 마무리해볼까?
Output:: 2

요일: 토요일
계절: 겨울
미세먼지: 나쁨
초미세먼지: 나쁨
날씨: 추움
최저기온: -1도
최고기온: 5도
일교차: 적음
시간대: 오전
나이: 36
성별: 남성
직업: 직장인
관심사: 캠핑, 청년1인가구
(미세먼지, 초미세먼지, 날씨, 시간대, 성별, 관심사) 따뜻하게 입을 겨울 옷차림
1. 따뜻한 니트와 레깅스, 그리고 따뜻한 스웨터로 겨울 옷차림을 완성해보자!
2. 따듯한 모자와 스카프로 꼭 감싸보자! 겨울이 시원하기도 하겠지?
Output:: 1
(미세먼지, 초미세먼지, 날씨, 시간대,